## API AlphaVantage - Intradiario

#### Importar las librerías necesarias

In [6]:
import requests
import json
import pandas as pd
import psycopg2

#### Leer la contraseña y el token desde la variable de entorno

In [7]:
#my_host = config('DATABASE_MYHOST')
#port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
#database_name = config('DATABASE_NAME')
#user = config('DATABASE_USER')
token = config('ALPHAVANTAGE_TOKEN')

NameError: name 'config' is not defined

#### Definir la función para obtener datos de AlphaVantage

In [4]:
def getIntra(symbol, interval, size): 
    function = 'TIME_SERIES_INTRADAY'
    url = 'https://www.alphavantage.co/query'
    parametros = {
        'function': function, 
        'symbol': symbol, 
        'interval': interval, 
        'outputsize': size, 
        'datatype': json, 
        'apikey': token
    }
    
    r = requests.get(url, params=parametros)
    data = r.json()['Time Series ('+interval+')']
    dataDF = pd.DataFrame.from_dict(data, orient='index') # Convertir a DataFrame
    dataDF = dataDF.astype('float')
    dataDF.index.name = 'Fecha'
    dataDF.columns = ['Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen']
    dataDF = dataDF.sort_values('Fecha', ascending=True)
    dataDF.index = pd.to_datetime(dataDF.index)
    return dataDF

acciones = ['AMZN', 'MELI', 'AAPL', 'GOOGL', 'MSFT'] # Lista de símbolos de acciones que se van a consultar


#### Crear e imprimir el Dataframe

In [5]:
combined_data = pd.DataFrame() # Inicializar un DataFrame vacío para combinar los datos

# Obtener y combinar los datos de las acciones en el DataFrame
for accion in acciones:
    data = getIntra(symbol=accion, interval='5min', size='compact')
    data['Simbolo'] = accion # Agregar una columna para el símbolo de la acción
    combined_data = pd.concat([combined_data, data])

combined_data = combined_data.round(3) # Redondear los valores a tres decimales

print(combined_data) # Imprimir el DataFrame combinado

NameError: name 'token' is not defined

#### Conectarse con Amazon Redshift

In [6]:
try:
    conn = psycopg2.connect(
        host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com", 
        port=5439, 
        database="data-engineer-database", 
        user="hvzambrana_coderhouse", 
        password=password
        )
    print("Conexión a la base de datos exitosa")

except Exception as e:
    print("Error al crear la conexión")
    print(e)

Error al crear la conexión
name 'password' is not defined


#### Crear la tabla (si no existe)

In [7]:
try:
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.alphavantage (
        simbolo VARCHAR(10) DISTKEY NOT NULL,
        fecha TIMESTAMP SORTKEY NOT NULL,
        apertura FLOAT(15) NULL,
        maximo FLOAT(15) NULL,
        minimo FLOAT(15) NULL,
        cierre FLOAT(15) NULL,
        volumen FLOAT(15) NULL
    );""")

    conn.commit()

except Exception as e:
    print("Error al crear la tabla")
    print(e)

Error al crear la tabla
name 'conn' is not defined


#### Insertar los datos en la tabla

In [48]:
try:
    cur = conn.cursor()
    
    print("Número de registros a insertar:", len(combined_data)) # Imprimir la cantidad de registros a insertar
    
    for index, row in combined_data.iterrows():
        try:
            cur.execute("""
            INSERT INTO hvzambrana_coderhouse.alphavantage (simbolo, fecha, apertura, maximo, minimo, cierre, volumen)
            VALUES (%s, %s, %s, %s, %s, %s, %s);""",
            (row['Simbolo'], index, row['Apertura'], row['Maximo'], row['Minimo'], row['Cierre'], row['Volumen']))

        except psycopg2.IntegrityError as e:
            print(f"Registro duplicado para fecha {index}. No se ha insertado.")

    conn.commit()
    print("Datos insertados con éxito")

except Exception as e:
    print("Error al insertar datos")
    print(e)

Número de registros a insertar: 500
Datos insertados con éxito


#### Cerrar la conexión a la base de datos

In [49]:
conn.close()